Here we will show you how to use stable-diffusion-webui to generate image with Lora and ControlNet support. The stable-diffusion-webui will be hosted at Amazon SageMaker Real-Time endpoint.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
region_name = boto3.session.Session().region_name

Logout from AWS public ECR to avoid the authentication token is expired.

In [ ]:
!docker logout public.ecr.aws

Build Docker image and push to ECR.

In [ ]:
!./build_and_push.sh $region_name

In [ ]:
model_data = f"s3://{bucket}/stable-diffusion-webui/data/model.tar.gz"
!touch dummy
!tar czvf model.tar.gz dummy
!rm dummy
!aws s3 cp model.tar.gz $model_data

In [ ]:
model_name = None
image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/all-in-one-ai-stable-diffusion-webui-inference-api:latest".format(
    account_id, region_name
)
base_name = sagemaker.utils.base_name_from_image(image_uri)

In [ ]:
import json

huggingface_models = [
    {
        "repo_id": "stabilityai/stable-diffusion-2-1",
        "filename": "v2-1_768-ema-pruned.ckpt",
        "name": "Stable-diffusion",
    },
    {
        "repo_id": "runwayml/stable-diffusion-v1-5",
        "filename": "v1-5-pruned.ckpt",
        "name": "Stable-diffusion",
    },
    {
        "repo_id": "lllyasviel/ControlNet",
        "filename": "models/control_sd15_canny.pth",
        "name": "ControlNet",
    },
    {
        "repo_id": "lllyasviel/ControlNet",
        "filename": "models/control_sd15_depth.pth",
        "name": "ControlNet",
    },
    {
        "repo_id": "lllyasviel/ControlNet",
        "filename": "models/control_sd15_normal.pth",
        "name": "ControlNet",
    },
    {
        "repo_id": "lllyasviel/ControlNet",
        "filename": "models/control_sd15_openpose.pth",
        "name": "ControlNet",
    },
    {
        "repo_id": "lllyasviel/ControlNet",
        "filename": "models/control_sd15_scribble.pth",
        "name": "ControlNet",
    },
    {
        "repo_id": "lllyasviel/ControlNet",
        "filename": "models/control_sd15_seg.pth",
        "name": "ControlNet",
    },
    {
        "repo_id": "andite/anything-v4.0",
        "filename": "anything-v4.5-pruned.safetensors",
        "name": "Lora",
    },
]

http_models = [
    {
        "uri": "https://civitai.com/api/download/models/7627",
        "filename": "2bNierAutomataLora_v2b.safetensors",
        "name": "Lora",
    },
    {
        "uri": "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml",
        "filename": "v2-1_768-ema-pruned.yaml",
        "name": "Stable-diffusion",
    },
]

model_environment = {
    "ckpt": "/tmp/models/Stable-diffusion/v1-5-pruned.ckpt",
    "huggingface_models": json.dumps(huggingface_models),
    "http_models": json.dumps(http_models),
    "generated_images_s3uri": f"s3://{bucket}/stable-diffusion-webui/generated",
    "embeddings_s3uri": f"s3://{bucket}/stable-diffusion-webui/embeddings",
    "hypernetwork_s3uri": f"s3://{bucket}/stable-diffusion-webui/hypernetwork",
}

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

model = Model(
    name=model_name,
    model_data=model_data,
    role=role,
    image_uri=image_uri,
    env=model_environment,
    predictor_cls=Predictor,
)

In [ ]:
instance_type = "ml.g4dn.2xlarge"
instance_count = 1

Here to be simplified, we use real-time inference. However, it has some limitations by nature. Real-time inference is suitable for workloads where payload sizes are up to 6 MB and need to be processed with low latency requirements in the order of milliseconds or seconds asynchronous inference is more suitable for workloads with large payload sizes and long inference processing times. Asynchronous inference also works for stable-diffusion-webui.

In [ ]:
predictor = model.deploy(
    instance_type=instance_type,
    initial_instance_count=instance_count,
    volume_size_in_gb=225,
    container_startup_health_check_timeout=1800,
)

Helper function for s3

In [ ]:
import boto3

s3_resource = boto3.resource("s3")


def get_bucket_and_key(s3uri):
    pos = s3uri.find("/", 5)
    bucket = s3uri[5:pos]
    key = s3uri[pos + 1 :]
    return bucket, key

LoRA (Low-Rank Adaptation of Large Language Models) models have become the standard to extend the Stable Diffusion models. Let's use Lora model to generate images.

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

inputs = {
    "task": "text-to-image",
    "model": "v1-5-pruned.ckpt",
    "txt2img_payload": {
        "enable_hr": False,
        "denoising_strength": 0,
        "firstphase_width": 0,
        "firstphase_height": 0,
        "hr_scale": 2,
        "hr_upscaler": "",
        "hr_second_pass_steps": 0,
        "hr_resize_x": 0,
        "hr_resize_y": 0,
        "prompt": "yorha no. 2 type b, 1girl, bangs, black blindfold, black dress, black gloves, black hairband, blindfold, blindfold removed, breasts, cleavage cutout, clothing cutout, commentary request, dress, gloves, hairband, half-closed eyes, hand up, highres, io (sinking=carousel), juliet sleeves, long sleeves, looking at viewer, medium breasts, mole, mole under mouth, nier (series), nier automata, no blindfold, parted lips, puffy sleeves, short hair, solo, thighhighs, turtleneck, upper body, white hair, bokeh <lora:2bNierAutomataLora_v2b:0.5>",
        "styles": [""],
        "seed": 2674865251,
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "sampler_name": "",
        "batch_size": 1,
        "n_iter": 1,
        "steps": 20,
        "cfg_scale": 7,
        "width": 512,
        "height": 512,
        "restore_faces": False,
        "tiling": False,
        "do_not_save_samples": False,
        "do_not_save_grid": False,
        "negative_prompt": "(worst quality, low quality:1.3)",
        "eta": 0,
        "s_churn": 0,
        "s_tmax": 0,
        "s_tmin": 0,
        "s_noise": 1,
        "override_settings": {},
        "override_settings_restore_afterwards": True,
        "script_args": [],
        "sampler_index": "DPM++ SDE Karras",
        "script_name": "",
        "send_images": True,
        "save_images": False,
        "alwayson_scripts": {},
    },
}

prediction = predictor.predict(inputs)

Process the generated images from Real-Time inference result.

In [ ]:
from PIL import Image
import io
from datetime import datetime

for image_uri in prediction["images"]:
    image_bucket, image_key = get_bucket_and_key(image_uri)
    image_object = s3_resource.Object(image_bucket, image_key)
    image = Image.open(io.BytesIO(image_object.get()["Body"].read()))
    image.show()
    image.save(datetime.now().strftime("%Y%m%d%H%M%S.jpg"))

ControlNet is a neural network structure to control diffusion models by adding extra conditions.

In [ ]:
from PIL import Image
import base64
import io


def encode_image_to_base64(image):
    with io.BytesIO() as output_bytes:
        image.save(output_bytes, format="JPEG")
        bytes_data = output_bytes.getvalue()

    encoded_string = base64.b64encode(bytes_data)

    base64_str = str(encoded_string, "utf-8")
    mimetype = "image/jpeg"
    image_encoded_in_base64 = (
        "data:" + (mimetype if mimetype is not None else "") + ";base64," + base64_str
    )
    return image_encoded_in_base64


def decode_base64_to_image(encoding):
    if encoding.startswith("data:image/"):
        encoding = encoding.split(";")[1].split(",")[1]
    try:
        image = Image.open(io.BytesIO(base64.b64decode(encoding)))
        return image
    except Exception as e:
        print(e)

In [ ]:
from PIL import Image

image = Image.open("./images/inference/ControlNet/bal-source.png").convert("RGB")

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

inputs = {
    "task": "text-to-image",
    "model": "v1-5-pruned.ckpt",
    "txt2img_payload": {
        "enable_hr": False,
        "denoising_strength": 0,
        "firstphase_width": 0,
        "firstphase_height": 0,
        "hr_scale": 2,
        "hr_upscaler": "",
        "hr_second_pass_steps": 0,
        "hr_resize_x": 0,
        "hr_resize_y": 0,
        "prompt": "ballon",
        "styles": [""],
        "seed": -1,
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "sampler_name": "",
        "batch_size": 1,
        "n_iter": 1,
        "steps": 20,
        "cfg_scale": 7,
        "width": 512,
        "height": 512,
        "restore_faces": False,
        "tiling": False,
        "do_not_save_samples": False,
        "do_not_save_grid": False,
        "negative_prompt": "",
        "eta": 0,
        "s_churn": 0,
        "s_tmax": 0,
        "s_tmin": 0,
        "s_noise": 1,
        "override_settings": {},
        "override_settings_restore_afterwards": True,
        "script_args": [],
        "sampler_index": "DPM++ SDE Karras",
        "script_name": "",
        "send_images": True,
        "save_images": False,
        "alwayson_scripts": {
            "controlnet": {
                "args": [
                    {
                        "enabled": True,
                        "module": "none",
                        "model": "control_sd15_canny [fef5e48e]",
                        "weight": 1,
                        "image": encode_image_to_base64(image),
                        "low_vram": False,
                        "processor_res": 64,
                        "threshold_a": 64,
                        "threshold_b": 64,
                        "guidance_start": 0,
                        "guidance_end": 1,
                        "guess_mode": False,
                    }
                ]
            }
        },
    },
}

prediction = predictor.predict(inputs)

Process the generated images from real-time inference result.

In [ ]:
from PIL import Image
import io
from datetime import datetime

for image_uri in prediction["images"]:
    image_bucket, image_key = get_bucket_and_key(image_uri)
    image_object = s3_resource.Object(image_bucket, image_key)
    image = Image.open(io.BytesIO(image_object.get()["Body"].read()))
    image.show()
    image.save(datetime.now().strftime("%Y%m%d%H%M%S.jpg"))

[Optional] Create auto-scaling group for SageMaker endpoint in case you want to scale it based on specific metrics automatically.

In [ ]:
def create_autoscaling_group_for_sagemaker_endpoint(
    endpoint_name, min_capcity=1, max_capcity=2, target_value=5
):
    # application-autoscaling client
    asg_client = boto3.client("application-autoscaling")

    # This is the format in which application autoscaling references the endpoint
    resource_id = f"endpoint/{endpoint_name}/variant/AllTraffic"

    # Configure Autoscaling on asynchronous endpoint down to zero instances
    response = asg_client.register_scalable_target(
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
        MinCapacity=min_capcity,
        MaxCapacity=max_capcity,
    )

    response = asg_client.put_scaling_policy(
        PolicyName=f"Request-ScalingPolicy-{endpoint_name}",
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
        PolicyType="TargetTrackingScaling",
        TargetTrackingScalingPolicyConfiguration={
            "TargetValue": target_value,
            "CustomizedMetricSpecification": {
                "MetricName": "ApproximateBacklogSizePerInstance",
                "Namespace": "AWS/SageMaker",
                "Dimensions": [{"Name": "EndpointName", "Value": endpoint_name}],
                "Statistic": "Average",
            },
            "ScaleInCooldown": 600,  # duration until scale in begins (down to zero)
            "ScaleOutCooldown": 300,  # duration between scale out attempts
        },
    )


create_autoscaling_group_for_sagemaker_endpoint(predictor.endpoint_name)

Resource cleanup.

In [ ]:
predictor.delete_endpoint()